In [6]:
type Value =
| AInt of int
| ASeq of Value list
module Decoder =
    let isInt num = System.Text.RegularExpressions.Regex(@"\d").IsMatch(string num)
    let rec numDecoder (state:string) (tokens:char list):(string*char list) =
        match tokens with
        | [] -> "", tokens
        | token :: tail when (isInt token) ->
            numDecoder (state + string token) tail
        | _ -> state, tokens

    let rec decoder (state:Value list) (tokens:char list):(Value list*char list) =
        match tokens with
        | [] -> state, tokens
        | '[' :: tail -> 
            let aSeq, aTail = decoder List.empty tail
            let aState = [ Value.ASeq aSeq ] |> List.append state
            decoder aState aTail
        | ']' :: tail ->
            state, tail
        | ',' :: tail -> decoder state tail
        | num :: tail -> 
            let aNumber, aTail = numDecoder (string num) tail
            let value = aNumber |> int |> Value.AInt
            decoder ([value] |> List.append state) aTail

    let decode (line:string):Value =
        line
        |> List.ofSeq
        |> decoder List.empty
        |> fst
        |> List.head

module Encoder =
    let rec toString (value:Value) =
        match value with
        | Value.AInt i -> string i
        | Value.ASeq aSeq ->
            let content =
                aSeq
                |> List.map toString
                |> String.concat ","
            "[" + content + "]"

module Comparer =
    type Outcome =
    | Good
    | Bad
    | Tie
    let rec compare (a:Value) (b:Value) =
        let intCompare ia ib =
            if ia = ib then
                Outcome.Tie
            elif ia - ib < 1 then 
                Outcome.Good
            else
                Outcome.Bad
        
        let listCompare (la:Value list) (lb:Value list) =
            match la, lb with
            | [],[] -> Outcome.Tie
            | [], _ -> Outcome.Good
            | _, [] -> Outcome.Bad
            | _ ->
            
            let outcome =
                Seq.zip la lb
                |> Seq.map (fun (va, vb) -> compare va vb)
                |> Seq.reduce (fun a b -> 
                    match a, b with
                    | Tie, _ -> b
                    | _, Tie -> a
                    | Good, _ -> Good
                    | Bad, _ | _, Bad -> Bad
                )
            match outcome with
            | Tie when la.Length > lb.Length ->
                Outcome.Bad
            | Tie when la.Length < lb.Length -> 
                Outcome.Good
            | _ ->
                outcome

        match a, b with
        | Value.AInt ia, Value.AInt ib -> 
            intCompare ia ib 
        | Value.ASeq la, Value.ASeq lb ->
            listCompare la lb
        | Value.AInt _, Value.ASeq lb ->
            listCompare [a] lb
        | Value.ASeq la, Value.AInt _ ->
            listCompare la [b]
        

let ResolutionFolder = __SOURCE_DIRECTORY__
let lines = 
    File.ReadLines( ResolutionFolder + "/input13.txt") 
    |> List.ofSeq
    |> List.filter ( String.IsNullOrEmpty >> not )

let signal =
    lines
    |> Seq.chunkBySize 2
    |> Seq.map ( Seq.map Decoder.decode )
    |> Seq.map List.ofSeq

let parsed =
    signal
    |> Seq.map ( Seq.map Encoder.toString )
    |> Seq.concat

let original = 
    lines
    |> Seq.filter ( String.IsNullOrEmpty >> not )

Seq.zip original parsed
|> Seq.mapi (fun idx (a, b) -> (a = b), $"{idx + 1}  {a}  |  {b}" )
|> Seq.filter (fst >> not)
|> display

signal
|> Seq.map ( fun it -> Comparer.compare it.[0] it[1] )
|> Seq.mapi (fun idx it -> idx + 1 ,it)
//|> Seq.filter (fun (_, it) -> match it with Comparer.Outcome.Good -> true | _ -> false)
//|> Seq.sumBy fst
|> display

let debug s =  
    s 
    |> List.map Decoder.decode 
    |> ( fun it -> Comparer.compare it.[0] it[1] ) 
    |> sprintf "%A"
    |> display

//[ "[[4,4],4,4]"; "[[4,4],4,4,4]" ] |> debug

//[ "[7,7,7,7]"; "[7,7,7]" ] |> debug


// 1 good
// 2 good
// 3 bad 
// 4 good
// 5 bad 
// 6 good
// 7 bad
// 8 bad


(empty)

index,Item1,Item2
0,1,Bad
1,2,Bad
2,3,Bad
3,4,Bad
4,5,Bad
5,6,Bad
6,7,Bad
7,8,Bad
8,9,Bad
9,10,Bad
